In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd   /content/drive/MyDrive/prueba_tenica_BC/
!ls

/content/drive/MyDrive/prueba_tenica_BC
 001-1007202-220301269555588250_pag1.json   model.png
 040-464031-220204640254303951_pag1.json    model_train
 bert_ds				   'OCR_reglas duras.ipynb'
 dataset				    train_model.ipynb
 EDA.ipynb				    train_tf_smallBert_expertBert.ipynb


In [3]:
!pip install textract-trp


In [4]:
import json
import trp
import re

In [5]:
def load_json (files):
  """
  Carga los datos de un archivo JSON.

  Args:
    Files: Archivo  JSON.

  Returns:
   Objeto tipo trp.
  """
  with open(files, 'r') as f:
      data = json.load(f)

  doc = trp.Document(data)

  return doc

In [6]:
def clean_text_words(text):
    """
    Elimina comas y puntos
    """
    clean_text = re.sub(r'[.,:]', '', text.strip())

    return clean_text

In [7]:
def format_date(text_date):
  """
  Le otorga formato a la fecha
  """
  date = text_date.split('de')
  month = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
        'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
  month_num = month.get(date[1].strip().lower())
  fecha_formateada = f"{date[-1][:5]}-{month_num:02d}-{int(date[0]):02d}"
  return fecha_formateada

In [45]:
def get_value(expresion, lines,flag_first = False):
  """
  A partir del objeto Document se obtiene una
  línea a través de una expresión regular

  Args:
    expresion: Expresión regular.
    lines: Línea de texto.
    flag_first: Match de la expresión regular

  Returns:
    Información del campo en el documento.
  """
  for line in lines:

    line_text = line.text
   # print(expresion)
    if line_text:
      match_re = list(map(lambda x : re.compile(x).search(line_text), expresion))
      coincidencia = list(set(match_re))[0]

      if coincidencia is not None and (flag_first == False):
        value = coincidencia.group(1)
        if value is not  None:
          return (value , line)
      elif coincidencia is not None and (flag_first != False):
        value = coincidencia.group(0)
        if value is not  None:
          return (value, line)
  return (None,None)

In [38]:
### Diccionario con las expresiones regulares
dict_expresion = {
    'Nro_Registro':[
    r'Nro Matrícula+(.*)' , r'Nro Matricula+(.*)',
    r'Nro\s*Matricula\s*:\s*(\d+[-\d]*)', r'Nro\s*Matricula\s*:\s*(\d{3}-\d{7})',
    r'Nro\s*de?\s*Matrícula:\s*(\d+[-\s]?\d+)', r'Nro\s*Matrícula:\s*(\S+)$' ],

    'fecha_impresion': [r'Impreso el\s+(.*)'],

    'departamento': [r'DEPTO:\s+(\w+)'],

    'municipio': [r'MUNICIPIO:\s+(\w+)'],

    'vereda': [r'VEREDA+(.*)'],

    'status_folio': [r"ACTIVO+(.*)|CERRADO+(.*)"],
}

### Orquestador de la  extracción

In [28]:
def selected_text(text,line_trp):
  """
  Se selecciona el txto o la línea
  si este está vacío

  Args:
    - text: texto extraído
    - line_trp: línea TRP

  Return: string

  """
  if text:
    return text
  elif line_trp:
    return line_trp.text
  else:
    return ''


In [29]:
def create_dict(path):
  """
  Orquesta la extracción de todos los campos

  Args:
    - path: ruta del OCR

  Return: diccioario con los campos y sus valores
  """
  res_dict = {}
  data_load = load_json(path)
  lines = data_load.pages[0].lines
  for key,exp in dict_expresion.items():
    text,line_trp = get_value(exp,lines)
    res_dict[key] = selected_text(text,line_trp)

  return res_dict

### extracción de ambos archivos

In [30]:
files = ['001-1007202-220301269555588250_pag1.json', '040-464031-220204640254303951_pag1.json']

In [44]:
for file in files:
  print(f'este es el archivo: {file}')
  print(create_dict(file))

este es el archivo: 001-1007202-220301269555588250_pag1.json
{'Nro_Registro': '001-1007202', 'fecha_impresion': '1 de Marzo de 2022 a las 02:29:10 PM', 'departamento': 'ANTIOQUIA', 'municipio': 'MEDELLIN', 'vereda': ': MEDELLIN', 'status_folio': 'ACTIVO'}
este es el archivo: 040-464031-220204640254303951_pag1.json
{'Nro_Registro': ': 040-464031', 'fecha_impresion': '4 de Febrero de 2022 a las 01:16:08 PM', 'departamento': 'ATLANTICO', 'municipio': 'BARRANQUILLA', 'vereda': ': BARRANQUILLA', 'status_folio': 'ACTIVO'}
